# Optimize shifted sphere function

## Initialize 

In [ ]:
import psopy as pso
import scipy.optimize as opt
import numpy as np
import matplotlib.cm as cm
from matplotlib import pyplot as plt 
import Functions_util as fs
import random as rd
import data
import time

rd.seed(2610)

## Study framework

We work in dimension 50 and 500, domain is 

In [ ]:


D = [50, 500]
L = [-100, 100]
n_try = 25
lD = len(D)
Dmax = max(D)

# generate random shifts, and initial values of x and functions
x_shifts = [fs.define_x_rand(count = n_try, dim = d) for d in D]
x_0s = [fs.define_x_rand(count = n_try, dim = d) for d in D]
s_fam = [fs.define_family_sphere(x_shifts[k]) for k in range(len(x_shifts))]

# gradient with unknown function
eps = 10**(-4) 
ro = 0.5
min_step = 10**(-6)

# with random data

x_opts = [np.empty((n_try, D[i]), dtype=float) for i in range(lD)]
f_opts = np.empty((lD, n_try), dtype=float)
n_runs = np.empty((lD, n_try), dtype=float)

for i in range(lD):
    dim = D[i]
    for t in range(n_try):
        print(dim, t)
        x_0 = x_0s[i][t]
        f = s_fam[i][t]
        (x_opts[i][t], f_opts[i][t], n_runs[i][t]) \
            = fs.gradient_method(dim, x_0, f, eps=eps, ro=ro, 
                                 termination_criterion = fs.termination_by_step, 
                                 min_step = min_step)

print(f_opts)
print(n_runs)

# with given data
x_shifts = [ np.array(data.sphere[0:d]) for d in D]
x_0s = [fs.define_x_rand(count = n_try, dim = d) for d in D]
s_fam = [fs.define_sphere(dim = D[k], x_shift = x_shifts[k]) for k in range(lD)]
x_opts = [np.empty((n_try, D[i]), dtype=float) for i in range(lD)]
f_opts = np.empty((lD, n_try), dtype=float)
n_runs = np.empty((lD, n_try), dtype=float)

for i in range(lD):
    dim = D[i]
    f = s_fam[i]
    for t in range(n_try):
        print(dim, t)
        x_0 = x_0s[i][t]
        (x_opts[i][t], f_opts[i][t], n_runs[i][t]) \
            = fs.gradient_method(dim, x_0, f, eps=eps, ro=ro, 
                                 termination_criterion = fs.termination_by_step, 
                                 min_step = min_step)

print(f_opts, n_runs)
print( x_opts )

# PSO with random data

# set constraints
const = [{'type':'ineq', 'fun': lambda x: x[i] - L[0]} for i in range(Dmax) ]
const.extend( [ {'type':'ineq', 'fun': lambda x: -x[i] + L[1]} for i in range(Dmax) ] )

# generate random shifts, and initial values of x and functions
n_pop = 50
n_try = 200

x_shifts = [fs.define_x_rand(count = n_try, dim = d) for d in D]
s_fam = [fs.define_family_sphere(x_shifts[k]) for k in range(lD)]

# test learning rates
rmin = 0.5
rmax = 5.5
ncheck = 11
step = (rmax - rmin) / (ncheck - 1)
rates = np.empty((ncheck, ncheck))
values = np.arange(rmin, rmax+0.1, step)

g = 0
for grate in values:
    l = 0
    for lrate in values:
        x_0s = [fs.define_x_rand(count = n_pop, dim = d) for d in D]
        i = (ncheck * l + g) % 25
        f = s_fam[0][i]
        myResult = pso.minimize(f, x_0s[0], 
                                options = {'max_iter':200,
                                          'verbose':False,
                                          'friction':0.5,
                                          'g_rate':grate,
                                          'l_rate':lrate},
                               constraints = const[0:50].extend(const[500:550]))
        rates[l, g] = myResult.fun
        print(l, g, rates[l, g])
        l += 1
    g += 1

rates1 = rates
plt.imshow(rates1, cmap=cm.gray)
plt.show()

# implement with data
n_pop = 500
n_try = 25
max_iter = 1000
g_rate = 5
l_rate = 0.2
friction = 0.5
max_velocity = 5
x_shifts = [ np.array( data.sphere[0:d] ) for d in D]
s_fam = [ fs.define_sphere( dim = D[k], x_shift = x_shifts[k] ) for k in range(lD) ]

dim = 50
start = time.time()
for t in range(n_try):
    print(dim, t)
    x_0s = fs.define_x_rand(count = n_pop, dim = dim)
    myResults = np.empty((25, 5), dtype = float)
    myXs = np.empty((25, dim), dtype=float)
    myResult = pso.minimize(s_fam[0], x_0s,
                            options = {'max_iter':max_iter,
                                       'verbose':False,
                                       'friction':friction,
                                       'max_velocity': max_velocity,
                                       'g_rate':g_rate,
                                       'l_rate':l_rate},
                            constraints = const[0:50].extend(const[500:550]))
    print(myResult)
    myResults[t,0] = myResult.fun
    myResults[t,1] = myResult.nit
    myResults[t,2] = myResult.nsit
    myResults[t,3] = myResult.status
    myResults[t,4] = myResult.success
    myXs[t] = myResult['x']
end = time.time()

# see results
print(myResults[:,0])
avgTime = (end - start) / t
print('average time =', avgTime)

# scaling in higher dimension

# test learning rates

n_pop = 50
n_try = 200

x_shifts = [fs.define_x_rand(count = n_try, dim = d) for d in D]
s_fam = [fs.define_family_sphere(x_shifts[k]) for k in range(lD)]

rmin = 0.5
rmax = 5.5
ncheck = 11
step = (rmax - rmin) / (ncheck - 1)
rates = np.empty((ncheck, ncheck))
values = np.arange(rmin, rmax+0.1, step)

g = 0
for grate in values:
    l = 0
    for lrate in values:
        x_0s = [fs.define_x_rand(count = n_pop, dim = d) for d in D]
        i = (ncheck * l + g) % 25
        f = s_fam[1][i]
        myResult = pso.minimize(f, x_0s[1], 
                                options = {'max_iter':200,
                                          'verbose':False,
                                          'friction':0.5,
                                          'g_rate':grate,
                                          'l_rate':lrate})
#                               constraints = const)
        rates[l, g] = myResult.fun
        print(l, g, rates[l, g])
        l += 1
    g += 1

rates2 = rates
plt.imshow(rates2, cmap=cm.gray)
plt.show()


# with real data

n_pop = 1000
n_try = 5
max_iter = 1500
g_rate = 7
l_rate = 0.2
friction = 0.9
max_velocity = 1

# real data
# we penalize the objective function to avoid checking constraints
x_shifts = [ np.array( data.sphere[0:d] ) for d in D]
s_fam = [ fs.define_sphere_pen( dim = D[k], x_shift = x_shifts[k] ) for k in range(lD) ]
dim = 500
start = time.time()
t = 0

for t in range(n_try):
    print(dim, t)
    x_0s = fs.define_x_rand(count = n_pop, dim = dim, lim = [-80, 80])
    myResults = np.empty((25, 5), dtype = float)
    myXs = np.empty((25, dim), dtype=float)
    myResult = pso.minimize(s_fam[1], x_0s,
                            options = {'max_iter':max_iter,
                                       'verbose':True,
                                       'friction':friction,
                                       'max_velocity': max_velocity,
                                       'g_rate':g_rate,
                                       'l_rate':l_rate})
    print(myResult)
    myResults[t,0] = myResult.fun
    myResults[t,1] = myResult.nit
    myResults[t,2] = myResult.nsit
    myResults[t,3] = myResult.status
    myResults[t,4] = myResult.success
    myXs[t] = myResult['x']
end = time.time()

print(myResults[:,0])

avgTime = (end - start) / t
print('average time =', avgTime)